# XGBoost.Dask in many threads

Sometimes we want to train many large XGBoost models in parallel.  We do so in this example with ...

1.  The `xgboost.dask` project to do large training runs
2.  Optuna to do hyper-parameter-optimization
3.  A thread pool, to run many of these in parallel
4.  Coiled to launch Dask clusters (but you could swap in your favorite Dask deployment technology as you like)

Using `xgboost.dask` from many threads tooks a couple of small tweaks across projects.  This notebook resulted in the following PRs and issues:

-  https://github.com/dask/distributed/issues/7377
-  https://github.com/dask/dask/pull/9723
-  https://github.com/dask/distributed/pull/7369
-  https://github.com/dmlc/xgboost/pull/8558 (mostly cosmetic, not necessary)
-  Also something in Coiled to allow package_sync to be thread-safe, should be released by 2022-12-07

In [1]:
FILEPATH="s3://prefect-dask-examples/nyc-uber-lyft/feature_table_fixed_upper_bound.parquet"

In [2]:
import datetime
import threading
from concurrent.futures import ThreadPoolExecutor

from distributed import Client, wait
import dask.dataframe as dd
from coiled import Cluster
import coiled

import optuna
from dask_ml.metrics import mean_squared_error as lazy_mse
import xgboost as xgb
from xgboost.dask import DaskDMatrix

from dask_ml.datasets import make_classification_df
from dask_ml.model_selection import train_test_split, KFold
from dask_ml.preprocessing import OneHotEncoder
import dask.array as da
import dask.dataframe as dd
from s3fs import S3FileSystem
from xgboost.core import XGBoostError
import numpy as np
import joblib
import pandas as pd
import dask

In [3]:
print("coiled:", coiled.__version__)
print("dask:", dask.__version__)
print("dask.distributed:", dask.distributed.__version__)
print("optuna:", optuna.__version__)
print("xgboost:", xgb.__version__)
print("coiled:", coiled.__version__)

coiled: 0.2.58
dask: 2022.12.1
dask.distributed: 2022.12.1
optuna: 3.1.0.dev
xgboost: 1.7.2
coiled: 0.2.58


### Train Model

In [4]:
# Here we subset data for cross-validation

def _make_cv(df, num_folds):
    frac = [1 / num_folds]*num_folds
    splits = df.random_split(frac, shuffle=True)
    for i in range(num_folds):
        train = [splits[j] for j in range(num_folds) if j != i]
        test = splits[i]
        yield train, test


In [5]:
def train_model(trial_number, study_params, n_splits=5, cluster_name = None):
    if cluster_name is None:
        thread_id = threading.get_ident()
        cluster_name = "xgb-nyc-taxi-" + str(thread_id)
    cluster = coiled.Cluster(
        worker_vm_types=["m6i.4xlarge"],
        scheduler_vm_types=["m6i.2xlarge"],
        package_sync=True, # copy local packages,
        name=cluster_name,
        shutdown_on_close=False,  # reuse cluster across runs
        show_widget=False,
        n_workers=10,
        use_best_zone=True,
        account="dask-engineering",
        backend_options={"region": "us-east-2", "spot": True, "spot_on_demand_fallback": True},
        scheduler_options={"idle_timeout": "10 minutes"},
        )

    print("starting run")
    with Client(cluster) as client:
        with client.as_current():
            # Load and pre-process the DataFrame
            ddf = dd.read_parquet(FILEPATH)
            categorical_vars = ddf.select_dtypes(include="category").columns.tolist()
            ddf = ddf.categorize(columns=categorical_vars)
            float_cols = ddf.select_dtypes(include="float").columns.tolist()
            ddf[float_cols] = ddf[float_cols].astype(np.float32).persist()  # Under the hood, XGBoost converts floats to `float32`

            val_scores = []

            for i, (train, test) in enumerate(_make_cv(ddf, n_splits)):
                print(f"Starting training run {i}")
                start = datetime.datetime.now()
                train = dd.concat(train)

                try:
                    assert all(train[c].cat.known for c in categorical_vars)
                    assert all(test[c].cat.known for c in categorical_vars)
                except Exception as e:
                    cluster.shutdown()
                    raise RuntimeError(f"Categorical_vars are not known")

                y_train = train['trip_time'].to_frame().persist()
                X_train = train.drop(columns=['trip_time']).persist()

                # Make the training data
                y_test = test['trip_time'].to_frame().persist()
                X_test = test.drop(columns='trip_time').persist()

                try:
                    print("Make dtrain")
                    dtrain = DaskDMatrix(client, X_train, y_train, enable_categorical=True)

                    # print("Make dtest")
                    # dtest = DaskDMatrix(client, X_test, y_test, enable_categorical=True)

                    print("Training model")

                    model = xgb.dask.train(
                        client,
                        {
                            'verbosity': 2,
                            'tree_method': 'hist', 
                            "objective": "reg:squarederror",
                            **study_params
                        },
                        dtrain,
                        num_boost_round=4,
                        evals=[(dtrain, "train")],
                    )

                    print("Make predictions")
                    # It's faster to run the prediction directly on X_test DataFrame
                    # We also need to confirm that predictions on dtest when it
                    # contains categoricals performs as expected
                    predictions = xgb.dask.predict(client, model, X_test)

                    print("Score the model")
                    score = lazy_mse(y_test.to_dask_array(lengths=True).reshape(-1,), 
                                     predictions.to_dask_array(lengths=True), squared=False,
                                    )
                    wait(score)  # Explicitly waiting avoids https://github.com/dask/distributed/issues/4612
                    print(f"rmse_score:  {score}")
                    val_scores.append(score)
                    print(f"val_scores:  {val_scores}")
                    print(f"Finished training run in:  {datetime.datetime.now() - start} seconds")

                except XGBoostError as e:
                    print(f"Trial {i} failed with {e}")

        return np.mean(val_scores)

In [6]:
train_options = dict(
    n_splits = 5 
)

In [7]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 125),
        'learning_rate': trial.suggest_float('learning_rate', 0.1, 0.9),
        'subsample': trial.suggest_float('subsample', 0.1, 1),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0, 1),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 3),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0, 1),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0, 1),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 0.5),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
    }
    rmse = train_model(
        trial_number=trial.number,
        study_params=params, 
        n_splits=train_options["n_splits"],
    )
    print(f"final mse:  {rmse}")
    return rmse

In [8]:
# create a single study


try:
    study = joblib.load("data/study.pkl")
    print(f"Best trial until now:  {study.best_trial.value}")
except Exception as e:
    study = optuna.create_study(study_name="nyc-taxi-study")    

executor = ThreadPoolExecutor(8)

futures = [
    executor.submit(study.optimize, objective, n_trials=1) for _ in range(50)
]


Best trial until now:  300.8710965752107
  Cloning https://github.com/optuna/optuna.git (to revision e8a010bb58aea943866e5f7addf0de953228de99) to /private/var/folders/b5/f_y899x168j7cs2m7szjld5c0000gn/T/pip-req-build-ydw42d4s


  Running command git clone --filter=blob:none --quiet https://github.com/optuna/optuna.git /private/var/folders/b5/f_y899x168j7cs2m7szjld5c0000gn/T/pip-req-build-ydw42d4s
  Running command git rev-parse -q --verify 'sha^e8a010bb58aea943866e5f7addf0de953228de99'
  Running command git fetch -q https://github.com/optuna/optuna.git e8a010bb58aea943866e5f7addf0de953228de99
  Running command git checkout -q e8a010bb58aea943866e5f7addf0de953228de99


  Resolved https://github.com/optuna/optuna.git to commit e8a010bb58aea943866e5f7addf0de953228de99
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for optuna: filename=optuna-3.1.0.dev0-py3-none-any.whl size=360987 sha256=0b5bf9a26c283c3b87c2e5f2474e8ea58f2573d545e3b47d600a4ca636838b5c
  Stored in directory: /private/var/folders/b5/f_y899x168j7cs2m7szjld5c0000gn/T/pip-ephem-wheel-cache-plcim0j_/wheels/51/b0/4b/c05d88297ef6716b5a87865bbbc77cf5b3aa7d959460a80772
Successfully built optuna
  Cloning https://github.com/optuna/optuna.git (to revision e8a010bb58aea943866e5f7addf0de953228de99) to /private/var/folders/b5/f_y899x168j7cs2m7szjld5c0000gn/T/pip-req-build-uuxq0xma


  Running command git clone --filter=blob:none --quiet https://github.com/optuna/optuna.git /private/var/folders/b5/f_y899x168j7cs2m7szjld5c0000gn/T/pip-req-build-uuxq0xma
  Running command git rev-parse -q --verify 'sha^e8a010bb58aea943866e5f7addf0de953228de99'
  Running command git fetch -q https://github.com/optuna/optuna.git e8a010bb58aea943866e5f7addf0de953228de99
  Running command git checkout -q e8a010bb58aea943866e5f7addf0de953228de99


  Resolved https://github.com/optuna/optuna.git to commit e8a010bb58aea943866e5f7addf0de953228de99
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for optuna: filename=optuna-3.1.0.dev0-py3-none-any.whl size=360987 sha256=3ea21dc25184879d2b13135eb3d6241e153528e1534f7bd3aae884e611301e3c
  Stored in directory: /private/var/folders/b5/f_y899x168j7cs2m7szjld5c0000gn/T/pip-ephem-wheel-cache-4v95uhae/wheels/51/b0/4b/c05d88297ef6716b5a87865bbbc77cf5b3aa7d959460a80772
Successfully built optuna
  Cloning https://github.com/optuna/optuna.git (to revision e8a010bb58aea943866e5f7addf0de953228de99) to /private/var/folders/b5/f_y899x168j7cs2m7szjld5c0000gn/T/pip-req-build-rs1kmr8p


...

[I 2023-01-04 19:48:53,714] Trial 207 finished with value: 302.8498302889251 and parameters: {'n_estimators': 84, 'learning_rate': 0.7630751421260675, 'subsample': 0.920003126584849, 'max_depth': 7, 'colsample_bytree': 0.8585015173352356, 'min_child_weight': 1, 'colsample_bynode': 0.8635856689649588, 'colsample_bylevel': 0.5787443996962995, 'reg_alpha': 0.17351592598363505, 'reg_lambda': 0.2778033889864235}. Best is trial 203 with value: 299.4087245471277.


rmse_score:  302.9084833770753
val_scores:  [303.0161876729516, 302.7797345712468, 302.89120080820277, 302.6535450151489, 302.9084833770753]
Finished training run in:  0:01:37.849846 seconds
final mse:  302.8498302889251
Make predictions
Score the model


[I 2023-01-04 19:49:13,199] Trial 206 finished with value: 305.84020827914685 and parameters: {'n_estimators': 84, 'learning_rate': 0.7065594124842038, 'subsample': 0.8894820014009499, 'max_depth': 7, 'colsample_bytree': 0.8538034126777354, 'min_child_weight': 1, 'colsample_bynode': 0.6862530734407966, 'colsample_bylevel': 0.5765223404373747, 'reg_alpha': 0.17737633438591682, 'reg_lambda': 0.30800476742283384}. Best is trial 203 with value: 299.4087245471277.


rmse_score:  306.7882647463431
val_scores:  [305.4282739038075, 305.4746246611275, 305.73526208188724, 305.7746160025689, 306.7882647463431]
Finished training run in:  0:01:44.966061 seconds
final mse:  305.84020827914685


2023-01-04 19:56:34,698 - distributed.deploy.cluster - WARNING - Failed to sync cluster info multiple times - perhaps there's a connection issue? Error:
Traceback (most recent call last):
  File "/Users/greghayes/Documents/projects/dask-xgboost-nyctaxi/.venv/lib/python3.9/site-packages/distributed/comm/tcp.py", line 498, in connect
    stream = await self.client.connect(
  File "/Users/greghayes/Documents/projects/dask-xgboost-nyctaxi/.venv/lib/python3.9/site-packages/tornado/tcpclient.py", line 275, in connect
    af, addr, stream = await connector.start(connect_timeout=timeout)
asyncio.exceptions.CancelledError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/asyncio/tasks.py", line 490, in wait_for
    return fut.result()
asyncio.exceptions.CancelledError

The above exception was the direct cause of the following exception:

Traceback (most recent call last

In [9]:
futures[-1].result()

In [10]:
study.best_params

{'n_estimators': 84,
 'learning_rate': 0.6672039422265867,
 'subsample': 0.8900505542603506,
 'max_depth': 7,
 'colsample_bytree': 0.8502263628107187,
 'min_child_weight': 1,
 'colsample_bynode': 0.8653618780464761,
 'colsample_bylevel': 0.5887080297021721,
 'reg_alpha': 0.17449357864171036,
 'reg_lambda': 0.297100190058965}

In [11]:
study.best_value

299.4087245471277

In [12]:
study.best_trial

FrozenTrial(number=203, state=TrialState.COMPLETE, values=[299.4087245471277], datetime_start=datetime.datetime(2023, 1, 4, 19, 27, 6, 946602), datetime_complete=datetime.datetime(2023, 1, 4, 19, 41, 44, 195661), params={'n_estimators': 84, 'learning_rate': 0.6672039422265867, 'subsample': 0.8900505542603506, 'max_depth': 7, 'colsample_bytree': 0.8502263628107187, 'min_child_weight': 1, 'colsample_bynode': 0.8653618780464761, 'colsample_bylevel': 0.5887080297021721, 'reg_alpha': 0.17449357864171036, 'reg_lambda': 0.297100190058965}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=125, log=False, low=50, step=1), 'learning_rate': FloatDistribution(high=0.9, log=False, low=0.1, step=None), 'subsample': FloatDistribution(high=1.0, log=False, low=0.1, step=None), 'max_depth': IntDistribution(high=9, log=False, low=3, step=1), 'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.0, step=None), 'min_child_weight': IntDis

In [13]:
len(study.trials)

208

In [14]:
joblib.dump(study, "data/study.pkl")

['data/study.pkl']

In [ ]:
f = futures[0].result()

In [ ]:
cluster.shutdown()

In [ ]:
dir(Client)

In [ ]:
dir(Cluster)